In [63]:
class Teacher:
    def __init__(self, name, list_grade, subject):
        self.name = name
        self.list_grade = list_grade
        self.subject = subject
    def get_name(self):
        return self.name
    def get_list_grade(self):
        return self.list_grade;
    def get_subject(self):
        return self.subject
    def to_string(self):
        return self.name + " " + self.subject
class Classes:
    def __init__(self, name):
        self.name = name
    def get_name(self):
        return self.name


class Subject:
    def __init__(self, name, all_time):
        self.name = name
        self.all_time = all_time

    def get_name(self):
        return self.name
    def get_all_time(self):
        return self.all_time

In [64]:
#import lib
import numpy as np
import json
from array import *
from tqdm import tqdm

In [65]:
#load sample data
inp_file = open('input/inp01.json', encoding='UTF-8')
sample_data = json.load(inp_file)
LIST_CLASSES = sample_data['Classes']
LIST_SUBJECT = sample_data['Subjects']
LIST_TEACHER = sample_data['Teachers']

TEACHERS = []
CLASSES = []
SUBJECTS = []

for i in range(len(LIST_CLASSES)):
    CLASSES.append(Classes(LIST_CLASSES[i]))
for i in range(len(LIST_SUBJECT)):
    SUBJECTS.append(Subject(LIST_SUBJECT[i]['name'], LIST_SUBJECT[i]['allTime']))
for i in range(len(LIST_TEACHER)):
    TEACHERS.append(Teacher(LIST_TEACHER[i]['name'], LIST_TEACHER[i]['listGrade'], LIST_TEACHER[i]['subject']))

for i in CLASSES:
    print(i.get_name())
for i in TEACHERS:
    print(i.get_subject())
for i in SUBJECTS:
    print(i.get_name())

6A
6B
6C
6D
7A
7B
7C
7D
8A
8B
8C
8D
9A
9B
9C
9D
Toán
Toán
Toán
Toán
Toán
Toán
Văn
Văn
Văn
Văn
Văn
Văn
Lịch sử và Địa lý
Lịch sử và Địa lý
Lịch sử và Địa lý
Lịch sử và Địa lý
Lịch sử và Địa lý
Khoa học tự nhiên
Khoa học tự nhiên
Khoa học tự nhiên
Khoa học tự nhiên
Khoa học tự nhiên
Tin học
Tin học
Tin học
Tin học
Tin học
Giáo dục công dân
Giáo dục công dân
Giáo dục công dân
Ngoại ngữ 1
Ngoại ngữ 1
Ngoại ngữ 1
Ngoại ngữ 2
Ngoại ngữ 2
Ngoại ngữ 2
Giáo dục thể chất
Giáo dục thể chất
Giáo dục thể chất
Nghệ thuật
Nghệ thuật
Nghệ thuật
Hoạt động trải nghiệm sáng tạo
Hoạt động trải nghiệm sáng tạo
Công nghệ và hướng nghiệp
Công nghệ và hướng nghiệp
Toán
Văn
Ngoại ngữ 1
Ngoại ngữ 2
Giáo dục công dân
Lịch sử và Địa lý
Khoa học tự nhiên
Tin học
Công nghệ và hướng nghiệp
Giáo dục thể chất
Nghệ thuật
Hoạt động trải nghiệm sáng tạo


In [66]:
#define constants
N_LESSON = 42
N_POPULATION = 100
MUTATION_RATE = 0.08
CROSSOVER_RATE = 0.6

In [67]:
#Step 1: Initial Population
def generate_individual():
    individual : list[list[Teacher]] = []
    for lesson in range(N_LESSON):
        individual.append([])
        for cls in range(len(CLASSES)):
            individual[lesson].append(TEACHERS[np.random.randint(0, len(TEACHERS))])
    return individual

def generate_population(populations: list):
    for index in range(N_POPULATION):
        populations.append(generate_individual())
    return populations

In [68]:
# step 2: build fitness function
def calculate_individual_score(individual: list[list[Teacher]]):
    score = 0

    # No teacher can hold two classes at the same time (same lesson)
    for lesson in range(N_LESSON):
        temp_set = set(individual[lesson])
        if len(temp_set) != len(CLASSES):
            score -= 1

    # One class in one subject not teach by two teacher
    for les1 in range(N_LESSON):
        for les2 in range(N_LESSON):
            for cls in range(len(CLASSES)):
                if les1 != les2 and individual[les1][cls].get_subject() == individual[les2][cls].get_subject():
                    score -= 1

    # Certainly, number lesson of each subject must be equal with define data
    num_lesson_per_class = []
    for cls in range(len(CLASSES)):
        num_lesson_per_class.append([])
        for sub in range(len(SUBJECTS)):
            num_lesson_per_class[cls].append(0)

    for lesson in range(N_LESSON):
        for cls in range(len(CLASSES)):
            for sub in range(len(SUBJECTS)):
                if individual[lesson][cls].get_subject() == SUBJECTS[sub].get_name():
                    num_lesson_per_class[cls][sub] += 1

    for cls in range(len(CLASSES)):
        for sub in range(len(SUBJECTS)):
            if num_lesson_per_class[cls][sub] != SUBJECTS[sub].all_time:
                score -= 1

     #
    for lesson in range(N_LESSON):
        for cls in range(len(CLASSES)):
            if int(CLASSES[cls].get_name()[0]) not in individual[lesson][cls].get_list_grade():
                score -= 1

    return score

def calculate_score_population(populations):
    population_score = []
    for index in range(N_POPULATION):
        score = calculate_individual_score(populations[index])
        population_score.append(score)
    return population_score

In [69]:
#Step 3: get fitness of population
def get_fitness_population(population_score, populations):
    max_score = max(population_fitness_score)
    fitness_individual = populations[population_score.index(max_score)]
    return max_score, fitness_individual

In [70]:
#Step 4: select one individual in population to crossover
def select_crossover_individual(partner_index, populations, model_parameter):
    rs_index = 0
    if len(populations) - partner_index <= model_parameter:
        rs_index = np.random.randint(0, len(populations) - model_parameter, 1)
    elif partner_index <= model_parameter:
        rs_index = np.random.randint(model_parameter, len(populations), 1)
    else:
        rs_index = np.random.randint(partner_index - model_parameter, partner_index + model_parameter, 1)
    return populations[int(rs_index)]


In [71]:
# Step 5: crossover, mutation function
def crossover(individual_a, individual_b):
    offspring_a = individual_a[11:15]
    offspring_b = individual_b[11:15]
    child_1 = np.concatenate((individual_a[0:11], offspring_b, individual_a[15:N_LESSON]))
    child_2 = np.concatenate((individual_b[0:11], offspring_a, individual_b[15:N_LESSON]))
    return child_1, child_2

def mutation(individual: list):
    point_a = int(np.random.randint(7, N_LESSON/2, 1))
    point_b = int(np.random.randint(N_LESSON/2, N_LESSON - 7, 1))
    offspring = individual[point_a:point_b]
    return np.concatenate((individual[0:point_a], offspring[::-1], individual[point_b: N_LESSON]))


In [72]:
# Step 6: crossover + mutation in population
def schedule_population(populations: list[list[Teacher]]):
    for index in range(N_POPULATION):
        if np.random.rand() <= CROSSOVER_RATE:
            select_individual = select_crossover_individual(index, populations, model_parameter=20)
            child_1, child_2 = crossover(populations[index], select_individual)
            populations.append(child_1)
            populations.append(child_2)
    for index in range(len(populations)):
        if np.random.rand() <= MUTATION_RATE:
            mutation_individual = mutation(populations[index])
            populations.pop(index)
            populations.append(mutation_individual)


In [73]:
#Step 7: natural selection population with size: N_POPULATION
def nature_selection(populations):
    population_score = calculate_score_population(populations)
    max_index = np.argsort(population_score)[::-1]
    rs_population = []
    for index in range(N_POPULATION):
        rs_population.append(populations[max_index[index]])
    return rs_population


In [74]:
#Step 7: Loop util get fitness
population_set = []
population_set = generate_population(population_set)
population_fitness_score = calculate_score_population(population_set)
FITNESS_SCORE, FITNESS_INDIVIDUAL = get_fitness_population(population_fitness_score, population_set)
TARGET_SCORE = -10000
for index in tqdm(range(5000)):
    schedule_population(population_set)
    new_population = nature_selection(population_set)
    population_fitness_score = calculate_score_population(new_population)
    FITNESS_SCORE, FITNESS_INDIVIDUAL = get_fitness_population(population_fitness_score, new_population)
    if FITNESS_SCORE > TARGET_SCORE:
        TARGET_SCORE = FITNESS_SCORE
    print('Loop {0}: => FITNESS_SCORE = {1}, TARGET_SCORE = {2}'.format(index, FITNESS_SCORE, TARGET_SCORE))
    population_set = new_population

print('Best solution: TARGET_SCORE = {0}'.format(TARGET_SCORE))

  0%|          | 1/5000 [00:01<1:57:12,  1.41s/it]

Loop 0: => FITNESS_SCORE = -2713, TARGET_SCORE = -2713


  0%|          | 2/5000 [00:03<2:14:44,  1.62s/it]

Loop 1: => FITNESS_SCORE = -2707, TARGET_SCORE = -2707


  0%|          | 3/5000 [00:04<2:16:46,  1.64s/it]

...

Loop 1982: => FITNESS_SCORE = -2641, TARGET_SCORE = -2641


 40%|███▉      | 1984/5000 [2:00:48<2:49:55,  3.38s/it]

Loop 1983: => FITNESS_SCORE = -2641, TARGET_SCORE = -2641


 40%|███▉      | 1984/5000 [2:00:51<3:03:43,  3.65s/it]


KeyboardInterrupt: 

In [91]:
import csv
map_date_of_week = {
    0: 'Monday',
    1: 'Tuesday',
    2: "Wednesday",
    3: 'Thursday',
    4: 'Friday',
    5: 'Saturday'
}
import os
def display_individual(individual: list[list[Teacher]]):
    print(os.getcwd())
    with open('data.csv', 'w', encoding='UTF-8', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['WeekDate','Lesson', 'Class', 'Teacher', 'Subject', 'TeacherCanTeach'])
        for lesson in range(N_LESSON):
            for cls in range(len(CLASSES)):
                writer.writerow([map_date_of_week[int(lesson / 7)],  7 if (lesson + 1) % 7 == 0 else (lesson + 1) % 7 , CLASSES[cls].get_name(),
                                individual[lesson][cls].get_name(), individual[lesson][cls].get_subject(),
                                 individual[lesson][cls].get_list_grade()])
display_individual(FITNESS_INDIVIDUAL)

E:\HeadWayDocs\workspace\GA-generate-timetable


In [ ]:
#Step 3: calculate score for population
#Step 4: get fitness of population: max score => store this individual to a variable
#Step 5: crossover
#Step 6: mutation
#Step 7: loop while until get fitness_score of population equal 0